In [1]:
import dash
import dash_bootstrap_components as dbc
import plotly.express as px
from dash import dcc,html,callback
import dash.dependencies as dd
import pandas as pd
import plotly.graph_objects as go
from dash.dependencies import Input, Output
import glob
import io
import base64
import  seaborn as sns
import matplotlib.pyplot as plt


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])


#----------------------cleaning------------------------------
def wrangle(filepath):
    df = pd.read_csv(filepath)

    # Replace specific values with 
    df.fillna(0, inplace=True)
    df.replace('_', 0, inplace=True)
    df.replace('-', 0, inplace=True)
   
    return df


data_pattern = "../data/Table_*.csv"

# Use glob to find the file paths
files = glob.glob(data_pattern)

# Create a list to store DataFrames
frames = []

# Iterate over the files and apply the wrangle function
for file in files:
    df = wrangle(file)
    # print(f"Processing {file}, df_shape: {df.shape}")
    frames.append(df)

# Access first dataframe
df1 = frames[0]

df2=frames[6]
df2['District'] = df2['District'].replace('Nyarugunge', 'Nyarugenge')
df2 = df2.pivot(index=['District', 'Season'], columns='Fertlizer apply', values='Agricultural area under fertirizer application').reset_index()
df2.rename(columns={'Inorganic Fertilizers': 'Agricultural area under Inorganic Fertilizers', 'Organic fertilizers': 'Agricultural area under Organic Fertilizers'}, inplace=True)
df2 = df2.fillna(0)

df3=frames[7]
df3["Agriculture land under modern irrigation"] = pd.to_numeric(df3["Agriculture land under modern irrigation"], errors='coerce')
# Define the custom sorting order for the "Season" column
season_order = ["A", "B", "C"]

# Sort the DataFrame by "District" and then "Season"
df3 = df3.sort_values(by=["District", "Season"], ascending=[True, False])

# Apply the custom sorting order for the "Season" column
df3["Season"] = pd.Categorical(df3["Season"], categories=season_order, ordered=True)
df3 = df3.sort_values(by=["District", "Season"])

# Reset the index to have a clean index after sorting
df3 = df3.reset_index(drop=True)

df4=frames[8]

file_path6 = "../data/Table_13 (1).csv"

# Read the Excel files into DataFrames
map= pd.read_csv(file_path6)

df5=frames[9]
# Rename the column 'District ' to 'District'
map = map.rename(columns={'District ': 'District'})

# Now you have renamed the column
df5['District']=df5['District'].str.lower()
# Merge df1 and df2 on the "District" column (many-to-one )
merged_df5 = pd.merge(df5,map, on='District', how='inner')

merged_df5['yield']=merged_df5['Average yield']
# transform
merged_df5['lat'] = pd.to_numeric(merged_df5['lat'], errors='coerce')
merged_df5['lon'] = pd.to_numeric(merged_df5['lon'], errors='coerce')
merged_df5['yield'] = pd.to_numeric(merged_df5['yield'], errors='coerce')

df6=frames[10]
df6['Season'] = df6['Season'].str.replace('Season', '')

# Agricultre inputes df10
df10=frames[11]
df10["District"].value_counts().head(10)
pesticides=df10.groupby("Pesticide").sum()
# Filter dataframe 
pesticidesA = df10[df10['Season'] == 'A'].groupby("Pesticide").sum().sort_values(by='Season', ascending=False)
pesticidesA_reset = pesticides.reset_index()
# Filter dataframe 
pesticidesB = df10[df10['Season'] == 'B'].groupby("Pesticide").sum().sort_values(by='Season', ascending=False)
# Reset the index to make 'pesticide' a regular column
pesticidesB_reset = pesticidesB.reset_index().sort_values(by='Percentage_use', ascending=False)


# Filter dataframe 
pesticidesC = df10[df10['Season'] == 'C'].groupby("Pesticide").sum().sort_values(by='Season', ascending=False)
# Reset the index to make 'pesticide' a regular column
pesticidesC_reset = pesticidesC.reset_index().sort_values(by='Percentage_use', ascending=False)

df7 = frames[11]
order = ['A', 'B', 'C']  # Adjust this list according to your desired order

# Convert 'Season' to Categorical with the custom order
frames[6]['Season'] = pd.Categorical(frames[6]['Season'], categories=order, ordered=True)

# Sort the DataFrame based on 'District' and the custom-ordered 'Season'
df7a = frames[6].sort_values(by=['District', 'Season'], ascending=[True, True])

df9a=frames[3]

df9a = df9a.sort_values(by=['Season', 'Value'])

df11=frames[12]
grouped_data = df11.groupby('source of fertilizers')['percectages'].mean()

# Now, you can sort the result
sorted_data = grouped_data.sort_values(ascending=False)

# Assuming df8 is a DataFrame with 'source of fertilizers' and 'percectages' columns
df11 = df11.sort_values('percectages', ascending=False)

# Merge df2 and df6 on 'District' and 'Season'
merged_df = pd.merge(df2, df6, on=['District', 'Season'], how='outer')

# Merge tmerged_df with df7 on 'District' and 'Season'
final_df = pd.merge(merged_df, df7a, on=['District', 'Season'], how='outer')

# Drop the repeated columns
final_df = final_df.loc[:, ~final_df.columns.duplicated()]

#frames[10].head()
df8=frames[2]
df8['District'] = df8['District'].astype(str)
df8.sort_values('District', inplace=True)
merge_df8=pd.concat([df8,map], axis=1)
# Assuming your DataFrame is named 'merged'
columns_to_keep = [0, 1, 2, 3, 5, 6]  # Specify the positions of columns to keep
merge_df8 = merge_df8.iloc[:, columns_to_keep]

df9=frames[1]

seasonA = df9[(df9['Season'] == 'A') & (df9['Year'] == 2022)]

df12=frames[13]

df12['Sowing Date'] = df12['Sowing Date'].astype('category')
#------------------------------------------------------------


#ACTION 2
# Group by district and season
grouped_data = merged_df5.groupby(['District', 'Season'])

# the row with the maximum yield for each group
max_yield_rows = grouped_data.apply(lambda x: x.loc[x['Average yield'].idxmax()])

# Create a list of unique seasons for the dropdown menu
seasons = merged_df5['Season'].unique()

#TIME SERIE GRAPH TO REPRESENT GVA(df4)

# Define the figure
fig = px.line(data_frame=df4, x='Year', y='GVA_in_price',
              color='Crop', markers=True, 
              title='GVA of Main Crops Over Time')
fig.update_layout(height=400) 


#AGRICULTURAL INPUTS(df ....)

# Sample data for farmers
data_farmers = {
    'farmers': ['Improved seeds', 'Organic fertilizer', 'Inorganic fertilizers', 'Pesticides'],
    'Season A': [33.1, 69.7, 38.6, 19.8],
    'Season B': [8.7, 57.9, 27.5, 15.9],
    'Season C': [26.1, 82.3, 71.5, 70.1]
}

# Sample data for land
data_land = {
    'Land': ['Improved seeds', 'Organic fertilizer', 'Inorganic fertilizer', 'Pesticides'],
    'Season A': [31.5, 65.4, 36.6, 19.2],
    'Season B': [17.0, 52.1, 26.0, 14.6],
    'Season C': [18.5, 85.6, 61.4, 92.1]
}

# Sample data for agricultural inputs
data_inputs = {
    'Agricultural inputs': ['Improved seeds', 'Organic fertilizer', 'Inorganic fertilizers', 'Pesticides'],
    'Season A': [30.9, 67.4, 30.3, 15.4],
    'Season B': [16.2, 55.2, 21.9, 11.9],
    'Season C': [13.2, 83.0, 42.6, 88.9]
}

# Create DataFrames
df_farmers = pd.DataFrame(data_farmers)
df_land = pd.DataFrame(data_land)
df_inputs = pd.DataFrame(data_inputs)

# ACTION 5: Source of improved seeds
df = pd.DataFrame(df9)

# Filter data for each season
seasonA = df[(df['Season'] == 'A') & (df['Year'] == 2022)]
seasonB = df[(df['Season'] == 'B') & (df['Year'] == 2022)]
seasonC = df[(df['Season'] == 'C') & (df['Year'] == 2022)]

# Create donut charts for each season
figA = px.pie(seasonA, 
              names='Source_of_input', 
              values='Percent_impoved_seed',
              hole=0.5,
              title='Improved Seed in Season A')

figB = px.pie(seasonB, 
              names='Source_of_input', 
              values='Percent_impoved_seed',
              hole=0.5,
              title='Improved Seed in Season B')

figC = px.pie(seasonC, 
              names='Source_of_input', 
              values='Percent_impoved_seed',
              hole=0.5,
              title='Improved Seed in Season C')

#pie charts for all seasons

# Define custom colors for the pie charts
custom_colors = px.colors.qualitative.Set3

# Create pie chart for Season A
fig_a = px.pie(pesticidesA_reset, values='Percentage_use', names='Pesticide', 
               title='Pesticides Percentage Use in Season A', 
               color_discrete_sequence=custom_colors)

# Create pie chart for Season B
fig_b = px.pie(pesticidesB_reset, values='Percentage_use', names='Pesticide', 
               title='Pesticides Percentage Use in Season B', 
               color_discrete_sequence=custom_colors)

# Create pie chart for Season C
fig_c = px.pie(pesticidesC_reset, values='Percentage_use', names='Pesticide', 
               title='Pesticides Percentage Use in Season C', 
               color_discrete_sequence=custom_colors)
#----------------------------------------------------------------------

#clustered bar charts

# Create the first grouped bar chart for df9a
fig1 = px.bar(df9a, x='Season', y='Value', color='Agricultural area', barmode='group',
              title='Agricultural Area by Season',
              labels={'Season': 'Season', 'Value': 'Value', 'Agricultural area': 'Agricultural Area'})
fig1.update_layout(width=600, height=400, xaxis_title='Season', yaxis_title='Value')

# Create the second ordinal bar chart for df11
fig2 = px.bar(df11, x='source of fertilizers', y='percectages', 
              title=' Source of Fertilizers',
              labels={'source of fertilizers': 'Source of Fertilizers', 'percectages': 'Percentage'})
fig2.update_layout(width=600, height=400, xaxis_title='Source of Fertilizers', yaxis_title='Percentage')


#---------------------------------------------------------------------------------------
# Create the treemap graph
fig_treemap = px.treemap(df12, 
                         path=['District', 'Season', 'Sowing Date'], 
                         values='Percentage of famers',
                         title='Percent sewed in Sewing Dates per district District',
                         labels={'Percentage of famers': 'Percent_sowed'},
                         hover_data={'Percentage of famers': ':.2f'})

# Set color to represent the total percentage in each box
fig_treemap.update_traces(marker=dict(colors=px.colors.sequential.Plasma))

#------------------------------------------------------------------------------------------------------------------

# Define the layout
app.layout = html.Div([
    dbc.Row([
            dbc.Col([
        # Place the title at the top
        dbc.Card([
                dbc.CardBody([
                html.Div("Seasonal Agriculture Survey_Project", style={'fontSize': '40px', 'textAlign': 'center', 'padding': '10px'}),
            ], className="border-start border-success border-5")
        ],className="text-center m-4"),
        ],width=5,className="mx-auto"),
    ]),
#----------------------the above is the title------------------------------------------
    dbc.Row([
        dbc.Col([
                html.Div([
                    dcc.Dropdown(
                id='season-dropdown',
                options=[{'label': season, 'value': season} for season in seasons],
                value=seasons[0],  # Set the default value to the first season
                multi=False
            ),
            dcc.Graph(
                id='yield-map',
            )], style={'border': '1px solid #ccc', 'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)'})
            ],width=6),
        dbc.Col([
            html.Div([dcc.Graph(
                id='time-series-graph',
                figure=fig  # Pass the figure directly to the dcc.Graph component
            )
            ], style={'border': '1px solid #ccc', 'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)'})

        ],width=6),
    ]),
    dbc.Row([
        dbc.Col(
            html.Div([dcc.Graph(
            id='farmers-radar',
            figure=px.line_polar(df_farmers.melt(id_vars='farmers', var_name='Season', value_name='Percentage'),
                                r='Percentage',
                                theta='farmers',
                                color='Season',
                                labels={'farmers': 'Input', 'Percentage': 'Percentage'},
                                title='Percentage of Farmers Using Agricultural Inputs by Season')
        )], style={'border': '1px solid #ccc', 'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)'}),width=4
        ),
        dbc.Col(
            html.Div([
                dcc.Graph(
            id='land-radar',
            figure=px.line_polar(df_land.melt(id_vars='Land', var_name='Season', value_name='Percentage'),
                                r='Percentage',
                                theta='Land',
                                color='Season',
                                labels={'Land': 'Input', 'Percentage': 'Percentage'},
                                title='Percentage of Land with Agricultural Inputs by Season')
        )], style={'border': '1px solid #ccc', 'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)'}),width=4
        ),
        dbc.Col(
            html.Div([
                dcc.Graph(
            id='inputs-radar',
            figure=px.line_polar(df_inputs.melt(id_vars='Agricultural inputs', var_name='Season', value_name='Percentage'),
                                r='Percentage',
                                theta='Agricultural inputs',
                                color='Season',
                                labels={'Agricultural inputs': 'Input', 'Percentage': 'Percentage'},
                                title='Percentage of Plots with Agricultural Inputs by Season')
        )], style={'border': '1px solid #ccc', 'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)'}),width=4
        )
    ]),
    dbc.Row([
        dbc.Col([
            html.Div([dcc.Graph(
                id='season-a-chart',
                figure=figA
            )], style={'border': '1px solid #ccc', 'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)'})
        ], width=4),  # Adjust the width as needed
        
        dbc.Col([
            html.Div([dcc.Graph(
                id='season-b-chart',
                figure=figB
            )], style={'border': '1px solid #ccc', 'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)'})
        ], width=4),  # Adjust the width as needed
        
        dbc.Col([
            html.Div([dcc.Graph(
                id='season-c-chart',
                figure=figC
            )], style={'border': '1px solid #ccc', 'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)'})
        ], width=4),  # Adjust the width as needed
    ]),
    dbc.Row([
        dbc.Col([
            html.Div([dcc.Graph(
                id='pie-chart-a',
                figure=fig_a
            )], style={'border': '1px solid #ccc', 'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)'})
        ], width=4),  # Adjust the width as needed

        dbc.Col([
            html.Div([dcc.Graph(
                id='pie-chart-b',
                figure=fig_b
            )], style={'border': '1px solid #ccc', 'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)'})
        ], width=4),  # Adjust the width as needed
        
        dbc.Col([
            html.Div([dcc.Graph(
                id='pie-chart-c',
                figure=fig_c
            )], style={'border': '1px solid #ccc', 'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)'})
        ], width=4),  # Adjust the width as needed
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id='bar-chart-1',
                figure=fig1
            )
        ]),  # Adjust the width as needed

        dbc.Col([
            dcc.Graph(
                id='bar-chart-2',
                figure=fig2
            )
        ], width=4),  # Adjust the width as needed
    ]),

    dbc.Row([
        dbc.Col([
            html.Div([dcc.Graph(
                id='treemap-chart',
                figure=fig_treemap
            )], style={'border': '1px solid #ccc', 'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)'})
        ], width=12),  # This column covers the whole width
    ]),
    
#---------------------end of row ---------------------------------------
])
#----------------------------------------------------
# Callback to update the map based on the selected season
@app.callback(
    dash.dependencies.Output('yield-map', 'figure'),
    [dash.dependencies.Input('season-dropdown', 'value')]
)
def update_map(selected_season):
    # Filter the data based on the selected season
    selected_data = max_yield_rows[max_yield_rows['Season'] == selected_season]

    # Create the scatter map for the selected season
    fig = px.scatter_mapbox(
        selected_data,
        lat="lat",
        lon="lon",
        color="Average yield",
        hover_data=["Average yield", "Crop", "Season", "District"],
        title=f"Highest Yielding Crop in {selected_season} for All Districts",
        # width=900,
        # height=900,
    )

    fig.update_layout(mapbox_style="open-street-map")

    return fig




if __name__ == '__main__':
    app.run_server(debug=True)
